In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import time
import warnings
from skinet.utils.athena_tools import *
warnings.filterwarnings("ignore")

In [2]:
# matplotlib 한글 나오게 하기

import matplotlib as mpl
import matplotlib.pylab as plt2 

plt2.rcParams["font.family"] = "Nanum"
plt2.rcParams["font.size"] = 12
plt2.rcParams["figure.figsize"] = (14, 4) 
mpl.rcParams['axes.unicode_minus'] = False 

In [3]:
from skinet.utils.athena_tools import *
conn = athena_connection()

In [4]:
df = pd.read_csv('유치원학원.csv')

In [5]:
magam = load_from_athena(conn, 'SELECT DISTINCT * FROM default.monthly_deadline_202010')

...tmp_db name = tmp_20201117_11_17_59_1d2903cb6d4fb3fbebe68aec7ae9475a07ff0971
Start creating table
drop table if exists default.tmp_20201117_11_17_59_1d2903cb6d4fb3fbebe68aec7ae9475a07ff0971
<< table not exist

        create  table         default.tmp_20201117_11_17_59_1d2903cb6d4fb3fbebe68aec7ae9475a07ff0971
        with (format='Parquet', parquet_compression='GZIP')
        as
        SELECT DISTINCT * FROM default.monthly_deadline_202010
..
>> Athena Query Success!!
>> Athena Query EXCUTION COMPLETE....
>> S3 FILE GENERATION SUCCESS!! (File count: 24)
Elapsed Time = 22.97
>> Start downloading table
aws s3 cp s3://adt-sandbox/dsc/tables/2fb94383-e157-432a-95a4-89bad5038adb /home/skinet/tmp/tmp_20201117_11_17_59_1d2903cb6d4fb3fbebe68aec7ae9475a07ff0971 --recursive
Elapsed Time = 33.23
>> Loading data
Dropping tmp db
<< Delete s3://adt-sandbox/dsc/tables/2fb94383-e157-432a-95a4-89bad5038adb
Removing tmp_dir!
Elapsed Time for downloading and loading = 14.61
Total Elapsed Time = 70.81

In [6]:
magam.shape

(766882, 61)

In [7]:
filtered = magam[(magam['sms_yn'] == 'Y') & (magam['adt1_yn'].isin(['Y', '', ' ']))]

In [8]:
filtered[['ctnum', 'itype_l', 'itype_s', 'sms_yn', 'adt1_yn', 'telf2']].head()

,ctnum,itype_l,itype_s,sms_yn,adt1_yn,telf2
0,216060780,1,242,Y,,01082120482
3,220007001,1,300,Y,Y,01030220044
5,218133638,1,180,Y,,01063030024
6,214075190,1,129,Y,,010-9453-1725
9,216002005,1,034,Y,,01093222063


In [9]:
filtered['ctnum'] = filtered['ctnum'].astype(str)

In [10]:
filtered['chk'] = "Y"

In [11]:
df['사업자등록번호'] = df['사업자등록번호'].fillna(0)
df['사업자등록번호'] = df['사업자등록번호'].astype(int)
df['사업자등록번호'] = df['사업자등록번호'].astype(str)

In [12]:
# 둘 다 and조건으로 n가 아니어야 함. 사업자등록번호 조건이 n이 아니고, filtered 조건도 n이 아니고.

In [13]:
# 여기에 사업자등록번호 조건 코드 넣기
df['사업자_중간'] = df['사업자등록번호'].str[3:5]

In [14]:
df['사업자_chk'] = np.where(df['사업자_중간'].isin(['82', '83']), "Y", "N")

In [15]:
df['계약번호'] = df['계약번호'].astype(str)

In [16]:
df_merged = df.merge(filtered[['ctnum', 'chk', 'telf2']], how='left', left_on='계약번호', right_on='ctnum')

In [17]:
df_merged.head()

,계약번호,변경요청번호,승인상태,경비상호,현상태,고객코드,업종,사업자등록번호,사업자_중간,사업자_chk,ctnum,chk,telf2
0,100000232,RT5389467,승인,자연숲유치원,경비,CU1379017,유치원어린이집,5138025358,80,N,NaN,NaN,NaN
1,100001062,RT5834195,승인,시립풍산어린이집,경비,CU1331842,유치원어린이집,1288023166,80,N,NaN,NaN,NaN
2,100001284,RN0001284,승인,미로미술,경비,CU1525844,유치원어린이집,0,,N,NaN,NaN,NaN
3,100001646,RT4876156,승인,청아어린이집,경비,CU1113913,유치원어린이집,5038209879,82,Y,NaN,NaN,NaN
4,100001791,RT4344225,승인,파크밸리어린이집,경비,CU1525907,유치원어린이집,0,,N,NaN,NaN,NaN


In [18]:
df_merged[(df_merged['사업자_chk']=='N') & (df_merged['chk'].isnull()==False)].groupby('업종').count()['계약번호']

업종
유치원어린이집    1665
학원         2968
Name: 계약번호, dtype: int64

In [22]:
df_merged['telf2'] = df_merged['telf2'].str.replace('-|/|' '|'  '', '', regex=True)
df_merged['telf2'] = np.where(df_merged['telf2'].str[0] != '0', "0"+df_merged['telf2'], df_merged['telf2'])
df_merged['telf2'] = df_merged['telf2'].str.replace(' ', '')
df_merged['telf2'] = df_merged['telf2'].str.replace('(', '')
df_merged['telf2'] = df_merged['telf2'].str.replace(')', '')

In [23]:
df_merged[(df_merged['사업자_chk']=='N') & (df_merged['chk'].isnull()==False)]

,계약번호,변경요청번호,승인상태,경비상호,현상태,고객코드,업종,사업자등록번호,사업자_중간,사업자_chk,ctnum,chk,telf2
6,100002788,RT4783525,승인,구립해솔어린이집,경비,CU1452978,유치원어린이집,5718000839,80,N,100002788,Y,01082291538
45,100025880,RT7303830,승인,시립장미어린이집(G),경비,CU1351407,유치원어린이집,1288047931,80,N,100025880,Y,01041441626
80,100031137,RT6918818,승인,예인성균관어린이집,경비,CU1340511,유치원어린이집,1418017731,80,N,100031137,Y,01024271121
93,100033187,RT4207393,승인,꼬마대통령어린이집(S),경비,CU1341717,유치원어린이집,2068018250,80,N,100033187,Y,01029816336
97,100033556,RT5776947,승인,구립여의동어린이집,경비,CU1342349,유치원어린이집,1078010169,80,N,100033556,Y,01089085806
104,100034268,RT7087602,승인,신미라유치원,경비,CU1331693,유치원어린이집,4108009215,80,N,100034268,Y,01036364848
116,100036151,RN0036151,승인,린덴바움유치원1층,경비,CU1348427,유치원어린이집,7678000366,80,N,100036151,Y,01085805903
123,100037317,RN0037317,승인,다솜유치원,경비,CU1365453,유치원어린이집,1178003888,80,N,100037317,Y,01065011523
142,100039956,RN0039956,승인,시립가장어린이집,경비,CU1368601,유치원어린이집,1288024791,80,N,100039956,Y,01046703549
165,100043709,RT6818493,승인,동원유치원,경비,CU1373289,유치원어린이집,1128902908,89,N,100043709,Y,01090052844


In [27]:
df_merged[(df_merged['사업자_chk']=='N') & (df_merged['chk'].isnull()==False)].to_csv('신사업지원팀요청_DTG_LMS_201117.csv', encoding='utf-8-sig')